<a href="https://colab.research.google.com/github/shiyuhu1933/EC-523-final-project/blob/main/iResNet_version_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import torch.nn

In [ ]:

def conv3x3(in_planes, out_planes, stride=1):
    conv3x3 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)
    return conv3x3

def conv1x1(in_planes, out_planes, stride=1):
    conv1x1 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)
    return conv1x1



class BasicBlock(nn.Module):
    exp_block = 1

    def __init__(self, inplanes, outplanes, stride=1, downsample=None, nm_layer=None, s_block=False, e_block=False, exd_bn0=False):
        super(BasicBlock, self).__init__()
        if nm_layer is None:
            nm_layer = nn.BatchNorm2d
        # When the stride != 1, conv and downsample layers
        if not s_block and not exd_bn0:
            self.bn0 = nm_layer(inplanes)

        self.conv1 = conv3x3(inplanes, outplanes, stride)
        self.bn1 = nm_layer(outplanes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(outplanes, outplanes)

        if s_block:
            self.bn2 = nm_layer(outplanes)

        if e_block:
            self.bn2 = nm_layer(outplanes)

        self.downsample = downsample
        self.stride = stride

        self.s_block = s_block
        self.e_block = e_block
        self.exd_bn0 = exd_bn0

    def forward(self, x):
        identity = x

        if self.s_block:
            out = self.conv1(x)
        elif self.exd_bn0:
            out = self.relu(x)
            out = self.conv1(out)
        else:
            out = self.bn0(x)
            out = self.relu(out)
            out = self.conv1(out)

        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)

        if self.s_block:
            out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out = out + identity

        if self.e_block:
            out = self.bn2(out)
            out = self.relu(out)

        return out

class iResNet(nn.Module):

    def __init__(self, block, layers, num_classes=num_classes, zero_init_residual=False, nm_layer=None, dropout_prob0=0.0):
        super(iResNet, self).__init__()
        if nm_layer is None:
            nm_layer = nn.BatchNorm2d
        self.inplanes = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nm_layer(64)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 64, layers[0], stride=2, nm_layer=nm_layer)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, nm_layer=nm_layer)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, nm_layer=nm_layer)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, nm_layer=nm_layer)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        if dropout_prob0 > 0.0:
            self.dp = nn.Dropout(dropout_prob0, inplace=True)
            print("The Probability of drop out is: ", dropout_prob0)
        else:
            self.dp = None

        self.fc = nn.Linear(512 * block.exp_block, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)



    def _make_layer(self, block, outplanes, blocks, stride=1, nm_layer=None):
        if nm_layer is None:
            nm_layer = nn.BatchNorm2d
        downsample = None
        if stride != 1 and self.inplanes != outplanes * block.exp_block:
            downsample = nn.Sequential(
                nn.MaxPool2d(kernel_size=3, stride=stride, padding=1),
                conv1x1(self.inplanes, outplanes * block.exp_block),
                nm_layer(outplanes * block.exp_block),
            )
        elif self.inplanes != outplanes * block.exp_block:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, outplanes * block.exp_block),
                nm_layer(outplanes * block.exp_block),
            )
        elif stride != 1:
            downsample = nn.MaxPool2d(kernel_size=3, stride=stride, padding=1)

        layers = []
        layers.append(block(self.inplanes, outplanes, stride, downsample, nm_layer, s_block=True))
        self.inplanes = outplanes * block.exp_block
        exd_bn0 = True
        for _ in range(1, (blocks-1)):
            layers.append(block(self.inplanes, outplanes, nm_layer=nm_layer, exd_bn0=exd_bn0))
            exd_bn0 = False

        layers.append(block(self.inplanes, outplanes, nm_layer=nm_layer, e_block=True, exd_bn0=exd_bn0))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)

        if self.dp is not None:
            x = self.dp(x)

        x = self.fc(x)

        return x


In [ ]:
model = iResNet(BasicBlock, [3,4,14,3]).cuda()